In [12]:
import random
import numpy as np
import pandas as pd

# Loss Function in Logistic Regression

## Random Variable

A coin toss is a random variable. We cannot possibly predict the result of the coin toss with certainty.

## 0) What are possible events of one coin toss?

- Head
- Tails

## 1) What is a fair coin?

If the probability of observing either event of a coin toss is 50%.

$P(Heads) = p = 0.5$<br>
$P(Tails) = 1-p = 0.5$

## 2) What is the probability of observing Heads when you toss a fair coin?

P(Kopf) = p = 0.5

The coin toss follows a **Bernoulli Distribution**. The **Bernoulli Distribution** is a probability distribution for random variables with two possible events.

Heads: y = 1 (like Survived = 1)<br>
Tails: y = 0 (like Survived = 0)

$$
P(y) = \begin{cases} p &\mbox{if } y = 1 \\
   1-p & \mbox{if } y = 0
   \end{cases}
$$

or (equivalently)

$$
P(y) = p^y * (1-p)^{1-y}
$$

## 3) What is the probability of observing the sequence of events (*Heads*, *Heads*) if a coin is tossed twice?

$$
P(Heads, Heads) = p * p = 0.25
$$

Equivalently it could be written as

$$
P(Heads, Heads) = P(Heads) * P(Heads)
$$

or as

$$
P(Heads, Heads) = (p^{y_1} * (1-p)^{1-y_1}) * (p^{y_2} * (1-p)^{1-y_2})
$$

oder 

$$
P(Heads, Heads) = \prod_{i=1}^2 p^{y_i} * (1-p)^{1-y_i}
$$

--> **The probability of observing a sequence of events can be written as the product of individual probabilities of bernoulli distributed random variables.**

Generally:

$$
P(y_1,...,y_n) = \prod_{i=1}^N p^{y_i} * (1-p)^{1-y_i}
$$

## 4) Let's turn around the question

**So far**: If we know *p*, what is the probability of observing a certain sequence of events?

**New**: If we observe a certain sequence of events in the real world, what does that tell us about *p*?

If we observe the sequence 

- Heads, Heads, Heads, Tails

and we do not know whether our coin is fair or not, what can we say about *p*?

- We will never know with certainty!!!
- Our best guess of what *p* is, is the value for which P(Heads, Heads, Heads, Tails) is highest.

We turn the probability of the sequence occuring under p

$$
P(Kopf, Kopf, Kopf, Zahl) = \prod_{i=1}^4 p^y_i * (1-p)^{1-y_i} = p * p * p * (1-p)
$$

into a likelihood-function of an estimate of *p* - *$\hat{p}$*

$$
L(\hat{p}) = \prod_{i=1}^4 \hat{p}^y_i * (1-\hat{p})^{1-y_i} = \hat{p} * \hat{p} * \hat{p} * (1-\hat{p})
$$

In [13]:
def calculate_likelihood(p_hat, sequence):
    '''
    Calculates the overall probability of observing a sequence of independent
    independent Bernoulli trial.
    
    Params:
    -------
    
    p_hat:    Estimate of p
    sequence: Observed sequence of events
    '''
    return (p_hat**np.array(sequence) * (1-p_hat)**(1-np.array(sequence))).prod()

In [14]:
# Werte für p die wir ausprobieren wollen
probabilities = np.linspace(0, 1, 9)

# Kopf, Kopf, Kopf, Zahl
sequence = [1, 1, 1, 0]

In [15]:
# Find the best value for p
for probability in probabilities:
    print(f'The likelihood of observing the sequence {sequence} if p is {probability} \
is {round(calculate_likelihood(probability, sequence), 3)}')

The likelihood of observing the sequence [1, 1, 1, 0] if p is 0.0 is 0.0
The likelihood of observing the sequence [1, 1, 1, 0] if p is 0.125 is 0.002
The likelihood of observing the sequence [1, 1, 1, 0] if p is 0.25 is 0.012
The likelihood of observing the sequence [1, 1, 1, 0] if p is 0.375 is 0.033
The likelihood of observing the sequence [1, 1, 1, 0] if p is 0.5 is 0.062
The likelihood of observing the sequence [1, 1, 1, 0] if p is 0.625 is 0.092
The likelihood of observing the sequence [1, 1, 1, 0] if p is 0.75 is 0.105
The likelihood of observing the sequence [1, 1, 1, 0] if p is 0.875 is 0.084
The likelihood of observing the sequence [1, 1, 1, 0] if p is 1.0 is 0.0


## 5) Which probability are we interested in in the Titanic project?

What are the events?<br>
**Events**: Survived, Died

P(Survived) = p<br>
P(Died) = 1-p

## 6) We are more interested in conditional probabilities

P(Survived|Age,Sex,Pclass) = p<br>
P(Died|Age,Sex,Pclass) = 1-p

## 7) The probabilities have to be estimated for all possible combinations of X

--> That is what the sigmoid function does (continuous - estimates **for all possible values**).<br>
--> The coefficients $b, w_1, w_2,...,w_n$ are estimated such that the likelihood is maximized.

$$
\hat{p} = \frac{1}{1+e^{-(b + w_1*Age + w_2*Sex + w_3*Pclass)}}
$$

## 8) Likelihood Function

$$
L(\hat{p}) = \prod_{i=1}^N \hat{p}^{y_i} * (1-\hat{p})^{1-y_i}
$$

## 9) Log-Likelihood Function

Mathematical optimization is easier if a function is additive.

- Taking the logarithm turns the product into a sum.
- The parameters that maximize a function are the same as the parameters that maximize the logarithm of the function.

$$
log(L(\hat{p})) = \sum_{i=1}^N y_i * log(\hat{p}) + (1-y_i) * log((1-\hat{p}))
$$

## 10) Loss Function

The loss function is just the negative log-likelihood function.

Maximizing a function is the same as minimizing its negative transformation.

$$
Loss = -log(L(\hat{p})) = - \sum_{i=1}^N y_i * log(\hat{p}) + (1-y_i) * log((1-\hat{p}))
$$

In [16]:
def log_loss_own(p_hat, sequence):
    '''
    Calculates the overall probability of observing a sequence of independent
    independent Bernoulli trial.
    
    Params:
    -------
    
    p_hat:    Estimate of p
    sequence: Observed sequence of events
    '''
    return - (np.log(p_hat)*np.array(sequence) + np.log((1-p_hat))*(1-np.array(sequence))).sum()

In [18]:
df = pd.read_csv("train.csv", index_col=0)

At this point you would go through the following steps:
* Assign X and y and train_test_split
* EDA
* Feature Engineering
* Training our model

In [24]:
X= df[["Pclass"]]
y= df["Survived"]


In [25]:
#Fit a model
from sklearn.linear_model import LogisticRegression 

In [26]:
m = LogisticRegression()

In [27]:
m.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [28]:
# w: slope
m.coef_

array([[-0.84371207]])

In [29]:
# b: position at x=0
m.intercept_

array([1.43255005])

In [30]:
# Now look at the log loss:
from sklearn.metrics import log_loss

In [31]:
# Prediction vs probability prediction:
m.predict_proba(X) #these are the probabilities of belonging to a particular class (0,1). The model optimises on this.



array([[0.7499951 , 0.2500049 ],
       [0.35690152, 0.64309848],
       [0.7499951 , 0.2500049 ],
       ...,
       [0.7499951 , 0.2500049 ],
       [0.35690152, 0.64309848],
       [0.7499951 , 0.2500049 ]])

In [32]:
log_loss(y, m.predict_proba(X))

0.6085336468537563

In [33]:
#define sigmoid func:
def sigmoid(x, b, w):
    return 1/(1+np.exp(-(b+x*w)))


In [40]:
# calculate the probability of a member of Pclass 1 surviving:
b= m.intercept_[0]
w= m.coef_[0]
Pclass = 1

In [41]:
sigmoid(Pclass,b,w)

array([0.64309848])